In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [4]:
pip install -U bitsandbytes


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install -q -U google-genai

In [2]:
import warnings
warnings.filterwarnings("ignore")

import re
import os
import json
import json_repair
import pandas as pd
from openai import OpenAI
from pydantic import BaseModel
from dotenv import load_dotenv
from pqdm.processes import pqdm
from datasets import load_dataset

import trl
import torch
import datasets
import transformers

import pandas as pd
from random import randint
from datasets import Dataset, load_dataset

from trl import SFTTrainer, setup_chat_format
from peft import LoraConfig, AutoPeftModelForCausalLM

import wandb
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline)

from huggingface_hub import login

import os
import json
from openai import OpenAI

# # 환경변수 로드
# load_dotenv("./credit-env")

In [3]:
print(f"PyTorch version       : {torch.__version__}")
print(f"Transformers version  : {transformers.__version__}")
print(f"TRL version           : {trl.__version__}")
print(f"CUDA available        : {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version      : {torch.version.cuda}")

PyTorch version       : 2.4.1+cu124
Transformers version  : 4.51.2
TRL version           : 0.16.1
CUDA available        : True
CUDA version      : 12.4


In [4]:
data_name = "daje/kotext-to-sql-v1-hard"
dataset = load_dataset(data_name, split="train")
dataset

README.md:   0%|          | 0.00/596 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'response', 'source', 'text', 'total_length', '__index_level_0__', 'translated_instruction', 'new_input'],
    num_rows: 2000
})

In [5]:
# 제로샷 테스트를 하기 위해 모델을 다운받고, 인퍼런스를 실행합니다. 
model_name = "Qwen/Qwen2.5-Coder-14B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    load_in_4bit=True 
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = ""
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": 'Task:라틴 아메리카/카리브해 지역의 인구가 783(7.5%)가 될 때, 아시아의 인구는 얼마가 될까요?\nSQL Table:CREATE TABLE table_22767 (\n    "Year" real,\n    "World" real,\n    "Asia" text,\n    "Africa" text,\n    "Europe" text,\n    "Latin America/Caribbean" text,\n    "Northern America" text,\n    "Oceania" text\n)\nQuery:'}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/47.5k [00:00<?, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [6]:
dataset = dataset.shuffle(seed=42)

dataset = dataset.train_test_split(test_size=0.1, seed=42)

In [7]:
dataset


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'response', 'source', 'text', 'total_length', '__index_level_0__', 'translated_instruction', 'new_input'],
        num_rows: 1800
    })
    test: Dataset({
        features: ['instruction', 'input', 'response', 'source', 'text', 'total_length', '__index_level_0__', 'translated_instruction', 'new_input'],
        num_rows: 200
    })
})

In [8]:
dataset["train"][0]

{'instruction': 'Which requirements do I need to meet for a CS-LSA degree ?',
 'input': 'CREATE TABLE program_course (\n    program_id int,\n    course_id int,\n    workload int,\n    category varchar\n)\n\nCREATE TABLE course_tags_count (\n    course_id int,\n    clear_grading int,\n    pop_quiz int,\n    group_projects int,\n    inspirational int,\n    long_lectures int,\n    extra_credit int,\n    few_tests int,\n    good_feedback int,\n    tough_tests int,\n    heavy_papers int,\n    cares_for_students int,\n    heavy_assignments int,\n    respected int,\n    participation int,\n    heavy_reading int,\n    tough_grader int,\n    hilarious int,\n    would_take_again int,\n    good_lecture int,\n    no_skip int\n)\n\nCREATE TABLE jobs (\n    job_id int,\n    job_title varchar,\n    description varchar,\n    requirement varchar,\n    city varchar,\n    state varchar,\n    country varchar,\n    zip int\n)\n\nCREATE TABLE student_record (\n    student_id int,\n    course_id int,\n    se

In [9]:
def get_chat_format(element):
    system_prompt = "너는 주어진 Schema와 Question을 보고 하는 SQL을 생성하는 AI Asisstant입니다."
    user_prompt = f"### instruction:{element['translated_instruction']} ### input:{element['input']} ### response:"
    return {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
            {"role": "assistant", "content": element["response"]+tokenizer.eos_token},  # 여기 닫는 괄호 추가
        ]
    }

# train과 test 데이터를 0.9와 0.1로 분할합니다.
dataset_train = dataset["train"].map(
    get_chat_format,
    batched=False,
    remove_columns=dataset["train"].features,  # 원하시면 제거
)


Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

In [10]:
dataset_train

Dataset({
    features: ['messages'],
    num_rows: 1800
})

In [11]:
dataset_train["messages"][0]

[{'content': '너는 주어진 Schema와 Question을 보고 하는 SQL을 생성하는 AI Asisstant입니다.',
  'role': 'system'},
 {'content': '### instruction:CS-LSA 학위를 받기 위해 충족해야 하는 요구 사항은 무엇인가요? ### input:CREATE TABLE program_course (\n    program_id int,\n    course_id int,\n    workload int,\n    category varchar\n)\n\nCREATE TABLE course_tags_count (\n    course_id int,\n    clear_grading int,\n    pop_quiz int,\n    group_projects int,\n    inspirational int,\n    long_lectures int,\n    extra_credit int,\n    few_tests int,\n    good_feedback int,\n    tough_tests int,\n    heavy_papers int,\n    cares_for_students int,\n    heavy_assignments int,\n    respected int,\n    participation int,\n    heavy_reading int,\n    tough_grader int,\n    hilarious int,\n    would_take_again int,\n    good_lecture int,\n    no_skip int\n)\n\nCREATE TABLE jobs (\n    job_id int,\n    job_title varchar,\n    description varchar,\n    requirement varchar,\n    city varchar,\n    state varchar,\n    country varchar,\n    zip int

In [12]:
print(text)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Task:라틴 아메리카/카리브해 지역의 인구가 783(7.5%)가 될 때, 아시아의 인구는 얼마가 될까요?
SQL Table:CREATE TABLE table_22767 (
    "Year" real,
    "World" real,
    "Asia" text,
    "Africa" text,
    "Europe" text,
    "Latin America/Caribbean" text,
    "Northern America" text,
    "Oceania" text
)
Query:<|im_end|>
<|im_start|>assistant



In [13]:
from trl import DataCollatorForCompletionOnlyLM
response_template = "<|im_start|>assistant\n"
# 만약 실제로 줄바꿈까지 포함되어 있다면 \n까지 넣어줘야 합니다.
# response_template = "<|start_header_id|>assistant<|end_header_id|>\n"

response_template_ids = tokenizer.encode(response_template, add_special_tokens=False)
collator = DataCollatorForCompletionOnlyLM(
    response_template_ids,
    tokenizer=tokenizer
)


In [14]:
peft_config = LoraConfig(
        lora_alpha=64,                            
        lora_dropout=0.05,                         # Lora 학습 때 사용할 dropout 확률을 지정합니다. 드롭아웃 확률은 과적합 방지를 위해 학습 중 무작위로 일부 뉴런을 비활성화하는 비율을 지정합니다.
        r=64,                                     # Lora의 저차원 공간의 랭크를 지정합니다. 랭크가 높을수록 모델의 표현력이 증가하지만, 계산 비용도 증가합니다.
        bias="none",                               # Lora 적용 시 바이어스를 사용할지 여부를 설정합니다. "none"으로 설정하면 바이어스를 사용하지 않습니다.
        target_modules=["q_proj", "o_proj",        # Lora를 적용할 모델의 모듈 리스트입니다.
                        "k_proj", "v_proj"
                        "up_proj", "down_proj",
                        "gate_proj",
                        ],
        task_type="CAUSAL_LM",                     # 미세 조정 작업 유형을 CAUSAL_LM으로 지정하여 언어 모델링 작업을 수행합니다.
)


args = TrainingArguments(
    output_dir="output_train1", # 모델 저장 및 허브 업로드를 위한 디렉토리 지정 합니다.
    num_train_epochs=5,                     # number of training epochs
    # max_steps=100,                        # 100스텝 동안 훈련 수행합니다.
    per_device_train_batch_size=2,          # 배치 사이즈 설정 합니다.
    gradient_accumulation_steps=2,          # 2스텝마다 역전파 및 가중치 업데이트합니다.
    gradient_checkpointing=False,           # 메모리 절약을 위해 그래디언트 체크포인팅 사용합니다.
    optim="adamw_torch_fused",              # 메모리 효율화할 수 있는 fused AdamW 옵티마이저 사용합니다.
    logging_steps=50,                       # 10스텝마다 로그 기록합니다.
    save_strategy="steps",                  # 매 에폭마다 체크포인트 저장합니다.
    learning_rate=5e-5,                     # 학습률 2e-4로 설정 (QLoRA 논문 기반)합니다.
    bf16=True,                              # 정밀도 설정으로 학습 속도 향상합니다.
    tf32=True,
    max_grad_norm=0.3,                      # 그래디언트 클리핑 값 0.3으로 설정합니다.
    warmup_ratio=0.03,                      # 워밍업 비율 0.03으로 설정 (QLoRA 논문 기반)합니다.
    lr_scheduler_type="constant",# 일정한 학습률 스케줄러 사용합니다.
    push_to_hub=False,                      # 훈련된 모델을 Hugging Face Hub에 업로드합니다.
    report_to="none",                       # wandb로 매트릭 관찰합니다.
)


trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset_train,
    peft_config=peft_config,
    # tokenizer=tokenizer,
    data_collator=collator,
)


Converting train dataset to ChatML:   0%|          | 0/1800 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1800 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1800 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1800 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [17]:
# trainer를 학습합니다.
trainer.train()

Step,Training Loss
50,0.486400
100,0.185100
150,0.111700
200,0.115300
250,0.102700
300,0.110800
350,0.090300
400,0.079500
450,0.066700
500,0.037400


TrainOutput(global_step=2250, training_loss=0.044881479375892215, metrics={'train_runtime': 5507.6165, 'train_samples_per_second': 1.634, 'train_steps_per_second': 0.409, 'total_flos': 7.626520275526164e+17, 'train_loss': 0.044881479375892215})

In [18]:
trainer.save_model("output_train1")

In [15]:
# 메모리 초기화
del model
del trainer
torch.cuda.empty_cache()

In [16]:
# 학습한 모델을 경로를 지정합니다.
save_dir = "/workspace/output_train1"
peft_model_id = f"{save_dir}"

# PEFT 어댑터를 통해 사전 학습된 모델을 로드합니다.
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
).to("cuda")

# 토크나이저 로드합니다.
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
tokenizer.padding_side = 'right'  
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [17]:
def get_chat_format_inference(element):
    system_prompt = "너는 주어진 Schema와 Question을 보고 하는 SQL을 생성하는 AI Asisstant입니다."
    user_prompt = f"### instruction:{element['translated_instruction']} ### input:{element['input']} ### response:"
    return {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    }

def apply_chat_format_inference(element):
    """
    1) get_chat_format_inference(element)를 호출해서
       messages를 생성한 뒤, Dataset에 저장할 dict로 반환합니다.
    """
    chat_format = get_chat_format_inference(element)  # get_chat_format은 원본 코드 그대로 사용
    return {
        "messages": chat_format["messages"]
    }


# train과 test 데이터를 0.9와 0.1로 분할합니다.
dataset_test = dataset["test"].map(
    apply_chat_format_inference,
    batched=False,
    remove_columns=dataset["test"].features,  # 원하시면 제거
)


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [18]:
from tqdm.auto import tqdm
output_list = []    
for idx in tqdm(range(len(dataset_test))):
    input_data = tokenizer.apply_chat_template(dataset_test[idx]["messages"], tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(input_data, return_tensors="pt").to("cuda")
    result = fine_tuned_model.generate(
            **inputs,                      # 토큰화된 입력 전달
            max_new_tokens=512,            # 생성할 최대 토큰 개수 설정
            temperature=0.1,               # 낮은 temperature로 예측 결과의 랜덤성을 낮춤 (결과가 일관됨)
            pad_token_id=tokenizer.eos_token_id  # 문장 끝을 나타내는 토큰 설정 (패딩 방지)
        )
    output = tokenizer.decode(
            result[0][len(inputs.input_ids[0]):],  # 입력 부분 토큰을 제외한 생성된 부분만 추출
            skip_special_tokens=True               # 특수 토큰을 제외하고 텍스트 변환
        )
    output_list.append(output)

  0%|          | 0/200 [00:00<?, ?it/s]

In [19]:
output

"SELECT d_icd_procedures.short_title FROM d_icd_procedures WHERE d_icd_procedures.icd9_code IN (SELECT t3.icd9_code FROM (SELECT t2.icd9_code, DENSE_RANK() OVER (ORDER BY COUNT(*) DESC) AS c1 FROM (SELECT admissions.subject_id, diagnoses_icd.charttime FROM diagnoses_icd JOIN admissions ON diagnoses_icd.hadm_id = admissions.hadm_id WHERE diagnoses_icd.icd9_code = (SELECT d_icd_diagnoses.icd9_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.short_title = 'malignant melanoma') AND DATETIME(diagnoses_icd.charttime) >= DATETIME(CURRENT_TIME(), '-5 year')) AS t1 JOIN (SELECT admissions.subject_id, procedures_icd.icd9_code, procedures_icd.charttime FROM procedures_icd JOIN admissions ON procedures_icd.hadm_id = admissions.hadm_id WHERE DATETIME(procedures_icd.charttime) >= DATETIME(CURRENT_TIME(), '-5 year')) AS t2 ON t1.subject_id = t2.subject_id WHERE t1.charttime < t2.charttime AND DATETIME(t2.charttime) <= DATETIME(t1.charttime, '+2 month') GROUP BY t2.icd9_code) AS t3 WHERE t3.c1 <= 4)"

In [20]:
dataset_test_input = [temp[1]["content"] for temp in dataset_test["messages"]]

In [21]:
final_df = pd.DataFrame({
    "inputs":dataset_test_input,
    "answers": dataset["test"]["response"],
    "predictions":output_list})

In [22]:
final_df.iloc[2]

inputs         ### instruction:항공편이 BOSTON에서 DENVER로 가는 항공사를 ...
answers        SELECT DISTINCT airline.airline_code FROM airl...
predictions    SELECT DISTINCT airline.airline_code FROM airl...
Name: 2, dtype: object

In [30]:
from dotenv import load_dotenv
load_dotenv("./credit-env")

True

In [31]:
from google import genai
from google.genai import types
from pydantic import BaseModel 

class json_output(BaseModel):
    reason: str
    score: bool

client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))
def evaluation_log(args):
    df, idx = args
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"""너는 주어진 SQL system prompt와 내가 만든 model이 SQL system prompt를 보고 만든 생성결과와 실제 정답을 너에게 전달해줄게. 이를 보고 모델이 알맞게 예측하였는지 평가하는 AI Asisstant입니다.
# SQL system prompt
{df['inputs'].iloc[idx]}
# model 생성결과
{df['predictions'].iloc[idx]}
# 실제 정답
{df['answers'].iloc[idx]}

# 평가 기준
- reason은 한국어로 작성하세요.
-score는 모델이 의미적으로 같은 결과를 조회하는 쿼리라면 True, 아니면 False로 평가하세요.
""",
        config=types.GenerateContentConfig(
        max_output_tokens=1000,  # Set the desired maximum number of output tokens
        temperature=0.0,
        response_mime_type="application/json",
        response_schema=json_output,
        ),
    )
    model_response = response.candidates[0].content.parts[0].text
    return model_response

from pqdm.processes import pqdm
args_list = [(final_df, idx) for idx in range(len(final_df))]
prediction_eval_result = pqdm(args_list, evaluation_log, n_jobs=5)

QUEUEING TASKS | :   0%|          | 0/200 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

PROCESSING TASKS | :   0%|          | 0/200 [00:00<?, ?it/s]

=(true | false)


COLLECTING RESULTS | :   0%|          | 0/200 [00:00<?, ?it/s]

In [32]:
import json_repair
json_repair_df = pd.DataFrame([json_repair.loads(temp) for temp in prediction_eval_result])
json_repair_df.tail(2)


,reason,score
198,모델 생성 결과는 질문이 작성된 날짜와 답변이 채택된 날짜 사이의 일수를 구하는 쿼...,False
199,"모델 생성 결과는 'malignant melanoma'를 질병으로 사용했지만, 실제...",False


In [33]:
print("score 평균: ", sum(json_repair_df['score']) / len(json_repair_df))

score 평균:  0.655
